### Import Libraries

In [ ]:
from ultralytics import YOLO
import cv2
import os
import matplotlib.pyplot as plt
import yaml
import logging
from typing import List, Dict
from datetime import datetime


### Load and train YOLO

In [ ]:

# Define paths (ensure these are correct paths)
data_yaml_path = "Augmented_Dataset/data.yaml"  # Path to your data.yaml file
output_dir = "train_results"  # Directory to save outputs
os.makedirs(output_dir, exist_ok=True)

# Model parameters
TARGET_CLASS = "chair"

# Load the YOLOv8 model for training
model = YOLO('yolov8n.pt')  # You can use 'yolov8s.pt' or 'yolov8m.pt' for larger models


# Load class names from data.yaml
with open(data_yaml_path, 'r') as file:
    data_yaml = yaml.safe_load(file)  # Load the content of the YAML file
class_names = data_yaml['names']  # This should give you a list of class names, e.g., ['chair']

# Training the model on your custom dataset
# The 'data' argument now uses the 'data.yaml' file for paths and class names
results = model.train(
    data=data_yaml_path,
    project=output_dir,
    save=True,
    verbose=True,
    epochs=100,               # Number of training epochs
    batch=8,                 # Batch size
    imgsz=1280,                # Image size
    lr0=1e-4,                 # Initial learning rate
    lrf=0.1,                  # Final learning rate
    optimizer="SGD",          # Optimizer
    weight_decay=0.00005,      # Regularization
    momentum=0.937,            # Momentum for SGD
    patience=20,               # Reduce patience
    freeze=5,                 # Freeze backbone layers
    mosaic=0.5,                 # Reduced mosaic due to small dataset

)

print(f"Detection results saved to: {output_dir}")


### Test the best model on unseen images

In [ ]:


def setup_logging(log_dir: str) -> None:
    """
    Set up logging configuration
    
    Args:
        log_dir (str): Directory to save log files
    """
    os.makedirs(log_dir, exist_ok=True)
    logging.basicConfig(
        level=logging.INFO, 
        format='%(asctime)s - %(levelname)s: %(message)s',
        filename=os.path.join(log_dir, 'chair_detection.log')
    )

def create_output_directories(base_dir: str) -> Dict[str, str]:
    """
    Create structured output directories
    
    Args:
        base_dir (str): Base directory for outputs
    
    Returns:
        Dict of directory paths
    """
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    output_root = f"chair_detection_outputs_{timestamp}"
    
    dirs = {
        'root': output_root,
        'logs': os.path.join(output_root, 'logs'),
        'predictions': os.path.join(output_root, 'predictions'),
        'annotated_images': os.path.join(output_root, 'annotated_images'),
        'detection_results': os.path.join(output_root, 'detection_results')
    }
    
    # Create all directories
    for dir_path in dirs.values():
        os.makedirs(dir_path, exist_ok=True)
    
    return dirs

def load_model_configuration(data_yaml_path: str) -> Dict:
    """
    Load and validate model configuration
    
    Args:
        data_yaml_path (str): Path to data configuration file
    
    Returns:
        Dict containing model configuration
    """
    try:
        with open(data_yaml_path, 'r') as file:
            data_yaml = yaml.safe_load(file)
        
        # Validate critical configurations
        assert 'names' in data_yaml, "Invalid YAML: Missing class names"
        
        # Validate target class exists
        target_class = "chair"
        assert target_class in data_yaml['names'], f"Target class '{target_class}' not in dataset"
        
        return {
            'data_yaml': data_yaml,
            'class_names': data_yaml['names'],
            'target_class': target_class
        }
    
    except Exception as e:
        logging.error(f"Configuration loading error: {e}")
        raise

def detect_chairs(
    model_path: str, 
    source: str, 
    output_dirs: Dict[str, str],
    data_yaml_path: str,
    conf_threshold: float = 0.25, 
    iou_threshold: float = 0.45
) -> List[Dict]:
    """
    Detect and count chairs in images or video
    """
    try:
        # Load model
        model = YOLO(model_path)
        
        # Load class names
        with open(data_yaml_path, 'r') as file:
            data_yaml = yaml.safe_load(file)
        
        class_names = data_yaml['names']
        target_class_index = class_names.index("chair")
        
        # Check if source is a video
        is_video = source.lower().endswith(('.mp4', '.avi', '.mov', '.mkv'))
        
        if is_video:
            # Open video
            input_video = cv2.VideoCapture(source)
            
            # Verify video is opened
            if not input_video.isOpened():
                print(f"ERROR: Cannot open video {source}")
                return []
            
            # Get video properties
            frame_width = int(input_video.get(cv2.CAP_PROP_FRAME_WIDTH))
            frame_height = int(input_video.get(cv2.CAP_PROP_FRAME_HEIGHT))
            fps = input_video.get(cv2.CAP_PROP_FPS)
            
            # Detailed video property logging
            print(f"Video Properties:")
            print(f"Width: {frame_width}")
            print(f"Height: {frame_height}")
            print(f"FPS: {fps}")
            
            # Precise video writer setup
            output_video_path = os.path.join(output_dirs['annotated_images'], 'chairs_detected.mp4')
            
            # Try different codecs
            codecs = ['mp4v', 'XVID', 'avc1']
            video_writer = None
            
            for codec in codecs:
                try:
                    fourcc = cv2.VideoWriter_fourcc(*codec)
                    video_writer = cv2.VideoWriter(
                        output_video_path, 
                        fourcc, 
                        fps, 
                        (frame_width, frame_height)
                    )
                    
                    if video_writer.isOpened():
                        print(f"Successfully created video writer with {codec} codec")
                        break
                except Exception as e:
                    print(f"Failed with {codec} codec: {e}")
            
            if video_writer is None:
                print("ERROR: Could not create video writer")
                return []
            
            processed_results = []
            frame_count = 0
            
            while True:
                ret, frame = input_video.read()
                if not ret:
                    break
                
                frame_count += 1
                
                # Perform detection
                results = model(frame, conf=conf_threshold, iou=iou_threshold)
                
                # Count chairs
                chair_count = sum(
                    1 for box in results[0].boxes 
                    if int(box.cls[0]) == target_class_index
                )
                
                # Annotate frame with detections
                annotated_frame = results[0].plot()
                
                # Add text overlay for chair count
                cv2.putText(
                    annotated_frame, 
                    f"# of chairs: {chair_count}", 
                    (15, 70),  # Adjust position as needed
                    cv2.FONT_HERSHEY_SIMPLEX, 
                    1.0,  # Font scale 
                    (0, 255, 0),  # Green color 
                    3  # Thickness
                )
                
                # Verify frame is correct
                if annotated_frame is None:
                    print(f"WARNING: Could not annotate frame {frame_count}")
                    continue
                
                # Write frame
                video_writer.write(annotated_frame)
                
                # Optional: Show progress
                if frame_count % 30 == 0:
                    print(f"Processed frame {frame_count}")
                
                processed_results.append({
                    'frame': frame_count,
                    'chair_count': chair_count
                })
            
            # Release resources
            input_video.release()
            video_writer.release()
            
            print(f"Video saved to: {output_video_path}")
            print(f"Total frames processed: {frame_count}")
            
            return processed_results
        
        # Image processing logic (unchanged from previous version)
        else:
            results = model.predict(
                source=source,
                conf=conf_threshold,
                iou=iou_threshold,
                save=False
            )
            
            processed_results = []
            results_csv_path = os.path.join(output_dirs['detection_results'], 'chair_counts.csv')
            
            with open(results_csv_path, 'w') as csv_file:
                csv_file.write("Image,Chair Count,Annotated Image Path\n")
                
                for result in results:
                    # Count chairs
                    chair_count = sum(
                        1 for box in result.boxes 
                        if int(box.cls[0]) == target_class_index
                    )
                    
                    # Annotate image
                    annotated_img = result.plot()
                    
                    # Add text for chair count
                    cv2.putText(
                        annotated_img, 
                        f"# of chairs: {chair_count}", 
                        (25, 160),
                        cv2.FONT_HERSHEY_SIMPLEX, 
                        5, 
                        (0, 255, 0), 
                        13
                    )
                    
                    # Save original prediction
                    pred_filename = f"pred_{os.path.basename(result.path)}"
                    pred_path = os.path.join(output_dirs['predictions'], pred_filename)
                    cv2.imwrite(pred_path, result.orig_img)
                    
                    # Save annotated image
                    annotated_filename = f"chairs_{os.path.basename(result.path)}"
                    annotated_path = os.path.join(output_dirs['annotated_images'], annotated_filename)
                    cv2.imwrite(annotated_path, annotated_img)
                    
                    # Log to CSV
                    csv_file.write(f"{result.path},{chair_count},{annotated_path}\n")
                    
                    # Log detection
                    logging.info(
                        f"Image: {result.path}, "
                        f"Chairs Detected: {chair_count}"
                    )
                    
                    processed_results.append({
                        'image_path': result.path,
                        'chair_count': chair_count,
                        'original_pred_path': pred_path,
                        'annotated_path': annotated_path
                    })
            
            return processed_results
    
    except Exception as e:
        logging.error(f"Detection error: {e}")
        import traceback
        traceback.print_exc()
        return []

def main():
    """
    Main execution function
    """
    try:
        # Paths configuration
        data_yaml_path = "Augmented_Dataset/data.yaml"
        best_model_path = "train_results/train/weights/best.pt"
        
        # Create output directories
        output_dirs = create_output_directories('.')
        
        # Setup logging
        setup_logging(output_dirs['logs'])
        
        # Load model configuration
        config = load_model_configuration(data_yaml_path)
        
        # Detect chairs (works for both images and video)
        results = detect_chairs(
            model_path=best_model_path,
            source='Augmented_Dataset/test/images',  # Replace with your video/image path
            output_dirs=output_dirs,
            data_yaml_path=data_yaml_path,
            conf_threshold=0.25,     # Slightly adjustable
            iou_threshold=0.45
        )
        
        # Print summary
        print(f"Total Items Processed: {len(results)}")
        print(f"Outputs organized in: {output_dirs['root']}")
        print(f"Detailed results logged in {os.path.join(output_dirs['logs'], 'chair_detection.log')}")
    
    except Exception as e:
        logging.error(f"Main process error: {e}")
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()